In [1]:
import gaussianRandomFeatures as GRF

In [2]:
from __future__ import division
import numpy as np, numpy.random as nr, numpy.linalg as nlg
import scipy as sp, scipy.linalg as slg, scipy.io as sio, scipy.sparse as ss
# import matplotlib.pyplot as plt

from multiprocessing import Pool

import time
import os, os.path as osp
import csv
import cPickle as pick
# import sqlparse as sql

import activeSearchInterface as ASI
import competitorsInterface as CI

import data_utils as du
import graph_utils as gu

import lapsvmp as LapSVM
import anchorGraph as AG

import IPython


In [3]:
t1 = time.time()
X0,Y0,classes = du.load_covertype(sparse=True, normalize=False)
X0 = du.bias_normalize_ft(X0)
print(time.time() - t1)

17.6202390194


In [56]:
rfc = GRF.RandomFeaturesConverter(X0.shape[0], 550, gammak=0.25)

In [5]:
# X2, Y2 = du.stratified_sample(X0, Y0, classes=[0,1], strat_frac=0.1, return_inds=False)
X2, Y2 = X0, Y0

In [57]:
t1 = time.time()
RX2 = rfc.getData(du.matrix_squeeze(X2.todense()).T).T
print(time.time() - t1)

18.5830509663


In [84]:
pi = Y2.sum() / Y2.shape[0]
sparse1 = True
sparse2 = False
verbose = True
prms1 = ASI.Parameters(pi=pi, sparse=sparse1, verbose=verbose)
prms2 = ASI.Parameters(pi=pi, sparse=sparse2, verbose=verbose)

wnprms1 = ASI.WNParameters(normalize=True, sparse=sparse1, verbose=verbose)
wnprms2 = ASI.WNParameters(normalize=True, sparse=sparse2, verbose=verbose)

In [85]:
kAS1 = ASI.linearizedAS(prms1)
kAS2 = ASI.linearizedAS(prms2)
wnAS1 = ASI.weightedNeighborAS(wnprms1)
wnAS2 = ASI.weightedNeighborAS(wnprms2)

In [86]:
n_init = 1
init_pt = Y2.nonzero()[0][nr.choice(len(Y2.nonzero()[0]),n_init,replace=False)]
init_labels = {p:1 for p in init_pt}

In [87]:
kAS1.initialize(X2, init_labels)
kAS2.initialize(RX2, init_labels)
wnAS1.initialize(X2, init_labels)
wnAS2.initialize(RX2, init_labels)

Constructing C
Time for constructing C: 0.527383089066
Inverting C
Time for inverse: 0.00472211837769
Done with the initialization.
Constructing C
Time for constructing C: 3.98571014404
Inverting C
Time for inverse: 1.09798288345
Done with the initialization.
Done with the initialization.
Done with the initialization.


In [88]:
hits1 = [n_init]
hits2 = [n_init]
hits3 = [n_init]
hits4 = [n_init]
K = 200
for _ in range(K):
#     idx1 = kAS1.getNextMessage()
#     kAS1.setLabelCurrent(Y2[idx1])
#     hits1.append(hits1[-1]+Y2[idx1])
    
    idx2 = kAS2.getNextMessage()
    kAS2.setLabelCurrent(Y2[idx2])
    hits2.append(hits2[-1]+Y2[idx2])

#     idx3 = wnAS1.getNextMessage()
#     wnAS1.setLabelCurrent(Y2[idx3])
#     hits3.append(hits3[-1]+Y2[idx3])
    
    idx4 = wnAS2.getNextMessage()
    wnAS2.setLabelCurrent(Y2[idx4])
    hits4.append(hits4[-1]+Y2[idx4])

Iter: 1, Selected: 473927, Hits: 1, Time: 0.292042
NNAS -- Iter: 1, Selected: 0, Hits: 1, Time: 4.712789
Iter: 2, Selected: 543735, Hits: 1, Time: 0.384036
NNAS -- Iter: 2, Selected: 572844, Hits: 1, Time: 4.806880
Iter: 3, Selected: 263663, Hits: 2, Time: 0.246169
NNAS -- Iter: 3, Selected: 299902, Hits: 1, Time: 4.451195
Iter: 4, Selected: 4957, Hits: 3, Time: 0.448452
NNAS -- Iter: 4, Selected: 543735, Hits: 1, Time: 4.314142
Iter: 5, Selected: 567152, Hits: 3, Time: 0.324596
NNAS -- Iter: 5, Selected: 4957, Hits: 2, Time: 3.936799
Iter: 6, Selected: 279412, Hits: 4, Time: 0.321688
NNAS -- Iter: 6, Selected: 567152, Hits: 2, Time: 4.034310
Iter: 7, Selected: 567273, Hits: 4, Time: 0.347020
NNAS -- Iter: 7, Selected: 567271, Hits: 2, Time: 4.021921
Iter: 8, Selected: 366847, Hits: 4, Time: 0.330612
NNAS -- Iter: 8, Selected: 473927, Hits: 2, Time: 3.855511
Iter: 9, Selected: 4865, Hits: 5, Time: 0.304152
NNAS -- Iter: 9, Selected: 4865, Hits: 3, Time: 3.996682
Iter: 10, Selected: 567

In [89]:
next_inds1 = [kAS2.unlabeled_idxs[idx] for idx in (-kAS2.f[kAS2.unlabeled_idxs]).argsort()[:100]]
next_inds2 = [wnAS2.unlabeled_idxs[idx] for idx in (-wnAS2.f).argsort()[:100]]
print(Y2[next_inds1].sum(), Y2[next_inds2].sum())

(37, 3)


In [68]:
RX2.shape

(550, 581012)